In [1]:
# Establecer directorio
import os 
os.chdir('c:\\users\\JOSE MENDOZA\\Dropbox\\Projects\\2023-2\\QLab\\PDs\\bases')

In [9]:
# Instalar mpi4py
! pip install mpi4py

### Crear código de ejemplo 1

In [1]:
%%writefile mpi_example1.py 
from mpi4py import MPI

comm = MPI.COMM_WORLD # all available processors to communicate 
rank = comm.Get_rank() # give ranks to processors, local variable for e/ processor
size = comm.Get_size() # each processor identifies the total processors 

print("Hello World from rank ", rank, " out of ", size, " processors ")

Writing mpi_example1.py


In [3]:
!mpiexec -n 8 python mpi_example1.py

Hello World from rank  4  out of  8  processors 
Hello World from rank  3  out of  8  processors 
Hello World from rank  2  out of  8  processors 
Hello World from rank  0  out of  8  processors 
Hello World from rank  6  out of  8  processors 
Hello World from rank  7  out of  8  processors 
Hello World from rank  1  out of  8  processors 
Hello World from rank  5  out of  8  processors 


#### Algunas preguntas
¿Cuántos procesadores tiene mi computadora?
¿Cuál es la diferencia entre procesadores y procesos?

### Crear código de ejemplo 2

In [6]:
%%writefile mpi_example2.py 
from mpi4py import MPI

rank = MPI.COMM_WORLD.Get_rank()
a = 6.0
b = 3.0
if rank == 0:
    print(a+b, "I am processor", rank)
if rank == 1:
    print(a*b, "I am processor", rank)
if rank == 2:
    print(max(a,b), "I am processor", rank)
if rank == 3:
    print(min(a,b),"I am processor", rank)

Overwriting mpi_example2.py


In [7]:
import time
start = time.time()
! mpiexec -n 4 python mpi_example2.py
end = time.time()
print(end - start)

18.0 I am processor 1
9.0 I am processor 0
6.0 I am processor 2
3.0 I am processor 3
0.17319154739379883


### Crear código de ejemplo 3a

In [8]:
%%writefile mpi_example3.py 
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
if rank == 0:
    data = {'a': 7, 'b': 3.14}
    comm.send(data, dest=1 , tag=1)
elif rank == 1:
    data = comm.recv(source=0 , tag=1)
    print("I am processor", rank)
    print(data)

Writing mpi_example3.py


In [9]:
start = time.time()
! mpiexec -n 4 python mpi_example3.py
end = time.time()
print(end - start)

I am processor 1
{'a': 7, 'b': 3.14}
0.17264819145202637


### Crear código de ejemplo 3b

In [10]:
%%writefile mpi_example3b.py 
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
if rank == 0:
    data1 = {'a': 7, 'b': 3.14}
    comm.send(data1, dest=1 , tag=1)
    data2 = {'Perú': 2, 'Australia': 0}
    comm.send(data2, dest=2 , tag=2)
elif rank == 1:
    data1 = comm.recv(source=0 , tag=1)
    print("I am processor", rank)
    print(data1)
elif rank == 2:
    data2 = comm.recv(source=0 , tag=2)
    print("I am processor", rank)
    print(data2)

Writing mpi_example3b.py


In [12]:
start = time.time()
! mpiexec -n 4 python mpi_example3b.py
end = time.time()
print(end - start)

I am processor 2
{'Perú': 2, 'Australia': 0}
I am processor 1
{'a': 7, 'b': 3.14}
0.1732940673828125


#### Algunas preguntas
¿Siempre se debe usar el mismo nombre para los datos enviados/recibidos?
¿Para qué sirven las etiquetas?

### Crear código de ejemplo 3c

In [13]:
%%writefile mpi_example3c.py 
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
if rank == 0:
    data = {'Perú': 2, 'Australia': 0}
    comm.send(data, dest=1 , tag=1)
    comm.send(data, dest=2 , tag=1)
elif rank == 1:
    data = comm.recv(source=0 , tag=1)
    print("I am processor", rank)
    print(data)
elif rank == 2:
    data = comm.recv(source=0 , tag=1)
    print("I am processor", rank)
    print(data)

Writing mpi_example3c.py


In [14]:
start = time.time()
! mpiexec -n 4 python mpi_example3c.py
end = time.time()
print(end - start)

I am processor 1
{'Perú': 2, 'Australia': 0}
I am processor 2
{'Perú': 2, 'Australia': 0}
0.17500567436218262


### Crear código de ejemplo 4a
La diferencia entre usar comandos en minúscula/mayúscula es que en mayúscula es mejor para enviar numpys más rápido

In [18]:
%%writefile mpi_example4.py 
from mpi4py import MPI

import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = np.arange(100000000, dtype=np.float)
    comm.Send(data, dest=1)
elif rank == 1:
    data = np.empty(100000000, dtype=np.float)
    comm.Recv(data, source=0)
    print(data)
    

Overwriting mpi_example4.py


In [19]:
start2 = time.time()
! mpiexec -n 4 python mpi_example4.py
end2 = time.time()
print(end2 - start2)

0.7782020568847656


Traceback (most recent call last):
  File "D:\1. Documentos\0. Bases de datos\0. Dofiles y Scripts\Diplomado_PUCP\Big Data\mpi_example4.py", line 12, in <module>
    data = np.empty(100000000, dtype=np.float)
                                     ^^^^^^^^
  File "C:\Users\canun\Lib\site-packages\numpy\__init__.py", line 319, in __getattr__
    raise AttributeError(__former_attrs__[attr])
AttributeError: module 'numpy' has no attribute 'float'.
`np.float` was a deprecated alias for the builtin `float`. To avoid this error in existing code, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations. Did you mean: 'cfloat'?
Traceback (most recent call last):
  File "D:\1. Documentos\0. Bases de datos\0. Dofiles y Scrip

### Crear código de ejemplo 4b
¿Qué sucede si uso los códigos en minúscula?

In [20]:
%%writefile mpi_example4b.py 
from mpi4py import MPI

import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = np.arange(100000000, dtype=np.float)
    comm.send(data, dest=1)
elif rank == 1:
    data = np.empty(100, dtype=np.float)
    data = comm.recv(source=0)
    print(data)

Writing mpi_example4b.py


In [21]:
start = time.time()
! mpiexec -n 4 python mpi_example4b.py
end = time.time()
print(end - start)

0.653907060623169


Traceback (most recent call last):
  File "D:\1. Documentos\0. Bases de datos\0. Dofiles y Scripts\Diplomado_PUCP\Big Data\mpi_example4b.py", line 12, in <module>
    data = np.empty(100, dtype=np.float)
                               ^^^^^^^^
  File "C:\Users\canun\Lib\site-packages\numpy\__init__.py", line 319, in __getattr__
    raise AttributeError(__former_attrs__[attr])
AttributeError: module 'numpy' has no attribute 'float'.
`np.float` was a deprecated alias for the builtin `float`. To avoid this error in existing code, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations. Did you mean: 'cfloat'?
Traceback (most recent call last):
  File "D:\1. Documentos\0. Bases de datos\0. Dofiles y Scripts\Diplomad

### Crear código de ejemplo 5

In [26]:
%%writefile mpi_example5.py 
from mpi4py import MPI
import matplotlib.pyplot as plt
import numpy as np
import time

def sim_rand_walks_parallel(n_runs):
    # Get rank of process and overall size of communicator:
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    # Start time:
    t0 = time.time()

    # Evenly distribute number of simulation runs across processes
    N = int(n_runs/size)

    # Simulate N random walks and specify as a NumPy Array
    r_walks = []
    for i in range(N):
        steps = np.random.normal(loc=0, scale=1, size=100)
        steps[0] = 0
        r_walks.append(100 + np.cumsum(steps))
    r_walks_array = np.array(r_walks)

    # Gather all simulation arrays to buffer of expected size/dtype on rank 0
    r_walks_all = None
    if rank == 0:
        r_walks_all = np.empty([N*size, 100], dtype='float')
    comm.Gather(sendbuf = r_walks_array, recvbuf = r_walks_all, root=0)

    # Print/plot simulation results on rank 0
    if rank == 0:
        # Calculate time elapsed after computing mean and std
        time_elapsed = time.time() - t0

        # Print time elapsed + simulation results
        print("Simulated %d Random Walks in: %f seconds on %d MPI processes"
                % (n_runs, time_elapsed, size))

        # Plot Simulations and save to file
        plt.plot(r_walks_all.transpose())
        plt.savefig("r_walk_nprocs%d_nruns%d.png" % (size, n_runs))

    return

def main():
    sim_rand_walks_parallel(n_runs = 10000)

if __name__ == '__main__':
    main()

Overwriting mpi_example5.py


In [27]:
! mpiexec -n 4 python mpi_example5.py
# Probar tmb cambiando el número de procesadores

Simulated 10000 Random Walks in: 0.110367 seconds on 4 MPI processes


In [ ]:
#Todo se manda al rank 0

In [27]:
%%writefile mpi_example7.py 
from mpi4py import MPI

# Get my rank and the number of ranks
rank = MPI.COMM_WORLD.Get_rank()
n_ranks = MPI.COMM_WORLD.Get_size()

if rank != 0:
    # All ranks other than 0 should send a message
    message = "Hello World, I'm rank {:d}".format(rank)
    MPI.COMM_WORLD.send(message, dest=0, tag=0)

else:
    # Rank 0 will receive each message and print them
    for sender in range(1, n_ranks):
        message = MPI.COMM_WORLD.recv(source=sender, tag=0)
        print(message)

Writing mpi_example7.py


In [28]:
! mpiexec -n 4 python mpi_example7.py
# Probar tmb cambiando el número de procesadores

Hello World, I'm rank 1
Hello World, I'm rank 2
Hello World, I'm rank 3


# Collective Communication

In [43]:
#Broadcasting a Python dictionary 

In [44]:
%%writefile mpi_example8.py 
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = {'key1' : [7, 2.72, 2+3j],
            'key2' : ( 'abc', 'xyz')}
else:
    data = None
data = comm.bcast(data, root=0)
print(data)

Overwriting mpi_example8.py


In [45]:
! mpiexec -n 4 python mpi_example8.py
# Probar tmb cambiando el número de procesadores

{'key1': [7, 2.72, (2+3j)], 'key2': ('abc', 'xyz')}
{'key1': [7, 2.72, (2+3j)], 'key2': ('abc', 'xyz')}
{'key1': [7, 2.72, (2+3j)], 'key2': ('abc', 'xyz')}
{'key1': [7, 2.72, (2+3j)], 'key2': ('abc', 'xyz')}


In [46]:
#Scattering Python objects:

In [47]:
%%writefile mpi_example9.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

if rank == 0:
    data = [(i+1)**2 for i in range(size)]
else:
    data = None
data = comm.scatter(data, root=0)
assert data == (rank+1)**2
print(data)

Overwriting mpi_example9.py


In [48]:
! mpiexec -n 4 python mpi_example9.py
# Probar tmb cambiando el número de procesadores

4
1
9
16


In [49]:
#Gathering Python objects:

In [52]:
%%writefile mpi_example10.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

data = (rank+1)**2
data = comm.gather(data, root=0)
if rank == 0:
    for i in range(size):
        assert data[i] == (i+1)**2
else:
    assert data is None
print(data)

Overwriting mpi_example10.py


In [53]:
! mpiexec -n 4 python mpi_example10.py
# Probar tmb cambiando el número de procesadores

None
None
None
[1, 4, 9, 16]


In [54]:
#Broadcasting a NumPy array:

In [55]:
%%writefile mpi_example11.py

from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = np.arange(100, dtype='i')
else:
    data = np.empty(100, dtype='i')
comm.Bcast(data, root=0)
for i in range(100):
    assert data[i] == i
print(data)

Writing mpi_example11.py


In [56]:
! mpiexec -n 4 python mpi_example11.py
# Probar tmb cambiando el número de procesadores

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26

In [ ]:
#Scattering NumPy arrays:

In [61]:
%%writefile mpi_example12.py

from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

sendbuf = None
if rank == 0:
    sendbuf = np.empty([size, 100], dtype='i')
    sendbuf.T[:,:] = range(size)
recvbuf = np.empty(100, dtype='i')
comm.Scatter(sendbuf, recvbuf, root=0)
assert np.allclose(recvbuf, rank)
print(recvbuf)
print(np.allclose(recvbuf, rank))

Overwriting mpi_example12.py


In [62]:
! mpiexec -n 4 python mpi_example12.py
# Probar tmb cambiando el número de procesadores

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
True
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
True
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
True
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
True


In [63]:
#Gathering NumPy arrays:

In [64]:
%%writefile mpi_example13.py
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

sendbuf = np.zeros(100, dtype='i') + rank
recvbuf = None
if rank == 0:
    recvbuf = np.empty([size, 100], dtype='i')
comm.Gather(sendbuf, recvbuf, root=0)
if rank == 0:
    for i in range(size):
        assert np.allclose(recvbuf[i,:], i)
print(recvbuf)

Writing mpi_example13.py


In [65]:
! mpiexec -n 4 python mpi_example13.py
# Probar tmb cambiando el número de procesadores

None
None
None
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
  2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
 [3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
  3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
  3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]]
